In [1]:
import time
import torch
from torch import nn
#from torch.optim import Adam
from torch.optim import SGD
#from torchtext.data import Field, BucketIterator
#from torchtext.datasets import SequenceTaggingDataset
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import copy
import torch.nn.functional as F

In [2]:
import pandas as pd
import numpy as np 
import csv

data_file = pd.read_csv('train', quoting=csv.QUOTE_NONE,sep=" ", names= ['index', 'word', 'tag'])
data_array = data_file.to_numpy()

In [3]:
data_file 

,index,word,tag
0,1,EU,B-ORG
1,2,rejects,O
2,3,German,B-MISC
3,4,call,O
4,5,to,O
...,...,...,...
204562,1,Swansea,B-ORG
204563,2,1,O
204564,3,Lincoln,B-ORG
204565,4,2,O


In [4]:
### store the lists of sentences into a big list 
 
def data_to_stream(data):
    data_stream = []
    i = 0 
    for index, word, tag in data:
        if index == 1:
            temp = []
            temp.append([index,word, tag])
            
        else:
            temp.append([index ,word,tag ])
            
        if ( (i+1 < len(data)) and data[i+1][0] == 1 ) or (i == len(data)-1):
            data_stream.append(temp)
        
        i += 1
            
    return data_stream 

In [5]:
train_data = data_to_stream(data_array)


In [6]:
def converter(data):
    train_word = []
    train_tag = []
    i = 0
    for index, word,tag in data:
        if index == 1:
            temp1 = []
            temp2 = []
            temp1.append(word)
            temp2.append(tag)
            
        else:
            temp1.append(word)
            temp2.append(tag)
        if ( (i+1 < len(data)) and data[i+1][0] == 1 ) or (i == len(data)-1):
            train_word.append(temp1)
            train_tag.append(temp2)
        i += 1
    return train_word, train_tag
    

In [7]:
train_word, train_tag = converter(data_array)

In [8]:
all_data_list = list(zip(train_word, train_tag ))

In [9]:
w_vocab = ['<pad>'] + sorted(set([str(word) for seq in train_word for word in seq])) + ['<unk>']
w_to_ix = {ch:i for i,ch in enumerate(w_vocab)}
ix_to_w = {i:ch for ch,i in w_to_ix.items()}

In [10]:
w_to_ix['<pad>']

0

In [11]:
t_vocab = ['<pad>'] + sorted(set([str(word) for seq in train_tag for word in seq]))
t_to_ix = {ch:i for i,ch in enumerate(t_vocab)}
ix_to_t = {i:ch for ch,i in t_to_ix.items()}

In [12]:
t_to_ix

{'<pad>': 0,
 'B-LOC': 1,
 'B-MISC': 2,
 'B-ORG': 3,
 'B-PER': 4,
 'I-LOC': 5,
 'I-MISC': 6,
 'I-ORG': 7,
 'I-PER': 8,
 'O': 9}

In [13]:
import torch
import torch.nn as nn

### define a function to  convert a review into a tensor vector 
def w2tensor(x, w_to_ix):
    ### keep max_review
    w_list = []
    for word in x:
        if word in w_to_ix:
            vec = w_to_ix[word]
        
        else:
            vec = w_to_ix['<unk>']
            
        
        w_list.append(vec)
        
    return w_list
    

In [14]:
import torch
import torch.nn as nn

### define a function to  convert a review into a tensor vector 
def t2tensor(x,t_to_ix):
    t_list = []
    for tag in x:
        if tag in t_to_ix:
            vec = t_to_ix[tag]
        
            t_list.append(vec)
        
    return t_list
    
    

In [15]:
### convert a review into [50,1,300] tensor, and map it with a label 
def transfer_to_index(mydataset):
    x_train, y_train = [], []
    for item in mydataset:
        words, tags = item[0], item[1]
        
        s_tensor = w2tensor(words,w_to_ix)
        tag_tensor =t2tensor(tags,t_to_ix) 
        
        x_train.append(s_tensor)
        y_train.append(tag_tensor)
        
    return x_train, y_train

In [16]:
x_train, y_train = transfer_to_index(all_data_list)

In [17]:
def pad(x, w_to_ix):
    ## get the length of each sentence
    X_lengths = [len(sentence) for sentence in x]
    # create an empty matrix with padding tokens
    pad_token = w_to_ix['<pad>']
    longest_sent = max(X_lengths)
    batch_size = len(x)
    padded_X = np.ones((batch_size, longest_sent)) * pad_token

    # copy over the actual sequences
    for i, x_len in enumerate(X_lengths):
        sequence = x[i]
        padded_X[i, 0:x_len] = sequence[:x_len]
        
    return padded_X
    

In [18]:
padded_X = pad(x_train, w_to_ix)
x_padded = torch.from_numpy(padded_X)

In [19]:
padded_Y = pad(y_train, t_to_ix)
y_padded = torch.from_numpy(padded_Y)

In [20]:
# prepare the data
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import TensorDataset, DataLoader

#DatasetTrain = TensorDataset(x_padded, y_padded)

In [21]:
dev_file = pd.read_csv('dev', quoting=csv.QUOTE_NONE, sep=" ", names= ['index', 'word', 'tag'])

dev_array = dev_file.to_numpy()

dev_word, dev_tag = converter(dev_array)

dev_data_list = list(zip(dev_word, dev_tag ))

x_dev, y_dev = transfer_to_index(dev_data_list)




In [22]:
dev_x = pad(x_dev, w_to_ix)
dev_y = pad(y_dev, t_to_ix)

dev_x = torch.from_numpy(dev_x)
dev_y = torch.from_numpy(dev_y)

x_padded = x_padded.type(torch.LongTensor) 
y_padded = y_padded.type(torch.LongTensor)

dev_x = dev_x.type(torch.LongTensor) 
dev_y = dev_x.type(torch.LongTensor)

In [23]:


DatasetTrain = TensorDataset(x_padded, y_padded)

DatasetDev = TensorDataset(dev_x, dev_y)

trainloader=torch.utils.data.DataLoader(DatasetTrain,batch_size=16,shuffle=True, drop_last=True, num_workers=0)
devloader=torch.utils.data.DataLoader(DatasetDev,batch_size=16,shuffle=False, num_workers=0)


In [24]:
class BiLSTM(nn.Module):
    def __init__(self, input_dim,embedding_dim, hidden_dim, output_dim,n_layers, bidirectional,dropout, pad_idx):
        
        super().__init__()   
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers = n_layers, 
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.elu = nn.ELU()
        
    def forward(self,X):
        #self.hidden = self.init_hidden()
        #pass text through embedding layer
        embedded = self.dropout(self.embedding(X))
        #pass embeddings into LSTM
        outputs, (hidden, cell) = self.lstm(embedded) 
        #we use our outputs to make a prediction of what the tag should be
        outputs = self.elu(outputs)
        outputs = self.fc(self.dropout(outputs))
        return outputs

In [25]:
w_vocab.index('<pad>')

0

In [26]:
INPUT_DIM = len(w_vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 128
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.33
PAD_IDX = w_vocab.index('<pad>')
#batch_size = 16 

model = BiLSTM(INPUT_DIM, 
               EMBEDDING_DIM, 
               HIDDEN_DIM, 
               OUTPUT_DIM, 
               N_LAYERS, 
               BIDIRECTIONAL, 
               DROPOUT, 
               PAD_IDX)

In [27]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)

In [28]:
model.apply(init_weights)

BiLSTM(
  (embedding): Embedding(23626, 100, padding_idx=0)
  (lstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (elu): ELU(alpha=1.0)
)

In [29]:
#device = torch.device('cuda:0')
#model.to(device)

In [30]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [31]:
### define optimizer fuction 
optimizer = SGD(model.parameters(), lr=0.05, momentum=0.9)

In [32]:
#corpus.tag_field.vocab.stoi[corpus.tag_field.pad_token]

In [33]:
### define loss function 
TAG_PAD_IDX = t_to_ix["<pad>"]

criterion = nn.CrossEntropyLoss(ignore_index =TAG_PAD_IDX)

criterion = criterion

In [34]:
### train the model 
def train(model, trainloader, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for w, t in trainloader:
        
        text = w
        #text = text.cuda()
        tags = t
        #tags = t.cuda()
        
        optimizer.zero_grad()
        
        predictions = model(text)
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        loss = criterion(predictions, tags)  
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        #scheduler.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(trainloader), epoch_acc / len(trainloader)

In [35]:
def evaluate(model,devloader , criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for w, t in devloader:

            text = w
            #text = text.cuda()
            tags = t
            #tags = t.cuda()
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(devloader), epoch_acc / len(devloader)

In [36]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
def categorical_accuracy(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

In [37]:
#device = torch.device("cuda:0")
#model.to(device)

In [38]:
N_EPOCHS = 20

best_train_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model,trainloader , optimizer, criterion, TAG_PAD_IDX)
    #valid_loss, valid_acc = evaluate(model, devloader, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    ### save the model 
    if train_loss < best_train_loss:
        best_train_loss = train_loss
        #torch.save(model.state_dict(), 'blstm__1.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

KeyboardInterrupt: 

In [55]:
def finder(x):
    token = []
    for w in x:
        if w in w_to_ix:
            vec = w_to_ix[w]
        else:
            vec = w_to_ix["<unk>"]
            
    return token 


In [56]:
#model.load_state_dict(torch.load('blstm1.pt'))
def tag_sentence(model, sentence, w_to_ix, t_to_ix):
    
    model.eval()
  
   # tokens = sentence
    # convert the word into index 

    numericalized_tokens = finder(sentence)
    # get the index of unkonwn word if have 
   # unk_idx = text_field.vocab.stoi[text_field.unk_token]
    # get the unknown word  
    #unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    ## convert it into tensor 
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1)
     # use the model to predict     
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return predicted_tags

In [58]:
## test on CPU 
device = torch.device("cpu")
model.to(device)

BiLSTM(
  (embedding): Embedding(23626, 100, padding_idx=0)
  (lstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (elu): ELU(alpha=1.0)
)

In [59]:
##create the nested list which contain the predicted tagas using our model 
whole_pred_tags = []

for s in dev_word:
    
    pred_tags = tag_sentence(model, s, w_to_ix, t_to_ix )
    
    whole_pred_tags.append(pred_tags)

In [63]:
### get the actual tags from dev 
#dev_tag

In [68]:
#whole_pred_tags

In [64]:
def flatten(t):
    return [item for sublist in t for item in sublist]

# flattenn the nested list 
whole_pred_tags_f = flatten(whole_pred_tags)

dev_actual_tags_f = flatten(dev_tag)

In [66]:
len（whole_pred_tags_f）

SyntaxError: invalid character in identifier (<ipython-input-66-3ae4d01dcac9>, line 1)

In [65]:
from sklearn.metrics import classification_report
labels =  ['B-LOC', 'B-MISC', 'B-ORG',"B-PER","I-LOC","I-MISC","I-ORG","I-PER"]
target = ['B-LOC', 'B-MISC', 'B-ORG',"B-PER","I-LOC","I-MISC","I-ORG","I-PER"]
print(classification_report(dev_actual_tags_f, whole_pred_tags_f, labels = labels, target_names =target ))

ValueError: Found input variables with inconsistent numbers of samples: [51578, 0]